In [1]:
import warnings
warnings.filterwarnings("ignore")
%load_ext autoreload
%autoreload 2
from qiskit_metal import Dict, MetalGUI, designs
from qiskit_metal.qlibrary.qubits.transmon_pocket import TransmonPocket
from qiskit_metal.qlibrary.qubits.transmon_pocket_6 import TransmonPocket6
from qiskit_metal.qlibrary.tlines.meandered import RouteMeander
from qiskit_metal.qlibrary.tlines.straight_path import RouteStraight
from qiskit_metal.qlibrary.terminations.launchpad_wb_coupled import LaunchpadWirebondCoupled
from qiskit_metal.qlibrary.terminations.short_to_ground import ShortToGround
from qiskit_metal.qlibrary.couplers.coupled_line_tee import CoupledLineTee
from qiskit_metal.qlibrary.user_components.left_coupler import LeftCoupler
from qiskit_metal.qlibrary.user_components.right_coupler import RightCoupler
from qiskit_metal.qlibrary.user_components.straightdown_coupler import StraightDownCoupler
from qiskit_metal.qlibrary.user_components.straightup_coupler import StraightUpCoupler

In [2]:
design = designs.DesignFlipChip({}, overwrite_enabled=True)
gui = MetalGUI(design)

In [5]:
design.delete_all_components()

In [17]:
design.get_chip_z('Q_chip')

'6um'

In [16]:
design.chips['C_chip'].size['size_x']='6mm'
design.chips['C_chip'].size['size_y']='6mm'
design.chips['Q_chip'].size['size_x']='4mm'
design.chips['Q_chip'].size['size_y']='4mm'
design.chips['Q_chip'].size['center_z']='6um'

# Resonator length

In [18]:
from qiskit_metal.analyses.em.cpw_calculations import guided_wavelength

def find_resonator_length(frequency, line_width, line_gap, N): 
    #frequency in GHz
    #line_width/line_gap in um
    #N -> 2 for lambda/2, 4 for lambda/4
    
    [lambdaG, etfSqrt, q] = guided_wavelength(frequency*10**9, line_width*10**-6,
                                              line_gap*10**-6, 750*10**-6, 200*10**-9)
    return str(lambdaG/N*10**3)+" mm"

In [19]:
resonator_length_4=[]
for i in range(3):
    a=find_resonator_length(7.2+i*0.2,10,6,4)
    a=a.strip('mm')
    a=float(a)-0.35+0.51-0.1+0.4-0.2-0.1
    
    resonator_length_4.append(a)
print(resonator_length_4)


[4.386627341935143, 4.2723941699207115, 4.164173270104566]


# Flipchip

In [20]:
cp0 = dict(down = dict(loc_W=1, loc_H=-1, pad_width='120um'))

# q0=[TransmonPocket6(design, 'Q0', options=dict(pos_x='-2000um',
#                                             pos_y='-2000um',
#                                             chip='Q_chip',
#                                             connection_pads=cp0))]
q0=[TransmonPocket6(design, 'Q0', options=dict(pos_x='-1300um',
                                            pos_y='-470um',
                                            chip='Q_chip',
                                            pad_width='355um',
                                            pad_height='65um',
                                            pad_gap='50um',
                                            pocket_width='650um',
                                            pocket_height='650um'))]

qubits=q0

q0x=float(qubits[0].options.pos_x.strip('um'))
q0y=float(qubits[0].options.pos_y.strip('um'))


c0 = StraightUpCoupler(design, "c0", options=dict(pos_x=f"{q0x-100}um",
                                              pos_y=f"{q0y+60}um",
                                              pad_width="150um",
                                              pad_height="65um",
                                              cpw_stub_height="200um",
                                              chip="C_chip"))

design.rebuild()
gui.rebuild()
gui.autoscale()

In [21]:
lpL=LaunchpadWirebondCoupled(design, 'lpL',
                                         options = dict(chip="C_chip",
                                         pos_x = '-1700um',
                                         pos_y = '1300um',
                                         lead_length = '30um'))
lpR=LaunchpadWirebondCoupled(design, 'lpR',
                                         options = dict(chip="C_chip",
                                         pos_x = '1700um',
                                         pos_y = '1300um',
                                         orientation='180',
                                         lead_length = '30um'))

In [22]:


stg=ShortToGround(design, 'stg', options=Dict(chip="C_chip",
                                                pos_x='-0.585mm',
                                                pos_y='0mm',
                                                orientation='270'))

In [23]:
fl0=RouteStraight(design, 'fl0',options=Dict(chip="C_chip", pin_inputs=Dict(start_pin=Dict(
                                                            component='lpL',
                                                            pin='tie'),
                                                            end_pin=Dict(
                                                            component='lpR',
                                                            pin='tie'))))


# Readout resonator

In [24]:
from collections import OrderedDict
jogs=OrderedDict()
def connect(component_name: str, component1: str, pin1: str, component2: str, pin2: str,
            length: str, start_jogged_extension: OrderedDict, end_jogged_extension: OrderedDict,
            spacing: str, asymmetry='0 um',  start_straight='0 um', end_straight='0 um', flip=False):
    """Connect two pins with a CPW."""
    myoptions = Dict(
        chip="C_chip",
        pin_inputs=Dict(
            start_pin=Dict(
                component=component1,
                pin=pin1),
            end_pin=Dict(
                component=component2,
                pin=pin2)),
        lead=Dict(
            start_straight=start_straight,
            end_straight=end_straight,
            start_jogged_extension=start_jogged_extension,
            end_jogged_extension=end_jogged_extension),
        total_length=length,
        fillet = '99.9um',
        meander=dict(asymmetry=asymmetry,
                     spacing=spacing))
    myoptions.meander.lead_direction_inverted = 'true' if flip else 'false'
    return RouteMeander(design, component_name, myoptions)

In [25]:
jogs1=OrderedDict()
jogs1[0]=['R','350um']
jogs1[1]=['R', '400um']
jogs1[2]=['L','465um']


In [26]:
#3.91mm
rr0=connect('rr0', 'c0', 'in','stg', 'short', 
            resonator_length_4[1],jogs1,jogs, 
            '400um','0um', '1460um', '100um')

# EPR Analysis

In [27]:
from qiskit_metal.analyses.quantization import EPRanalysis
eig_rd = EPRanalysis(design, "hfss")
hfss = eig_rd.sim.renderer

In [28]:
hfss.start()

INFO 06:05PM [connect_project]: Connecting to Ansys Desktop API...
INFO 06:05PM [load_ansys_project]: 	Opened Ansys App
INFO 06:05PM [load_ansys_project]: 	Opened Ansys Desktop v2018.0.0
INFO 06:05PM [load_ansys_project]: 	Opened Ansys Project
	Folder:    C:/Users/kimje/Documents/Ansoft/
	Project:   flipchip_distance
INFO 06:05PM [connect_design]: 	Opened active design
	Design:    flip7um_V5-1 [Solution type: Eigenmode]
INFO 06:05PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.HfssEMSetup'>)
INFO 06:05PM [connect]: 	Connected to project "flipchip_distance" and design "flip7um_V5-1" 😀 



True

In [29]:
hfss.activate_ansys_design("flip6um_V5", 'eigenmode')
hfss.render_design(['Q0','c0','lpL','lpR','stg','rr0','fl0'],[],
                    box_plus_buffer=False)

06:05PM 54s WARNING [activate_ansys_design]: The design_name=flip6um_V5 was not in active project.  Designs in active project are: 
['flip7um_V5-1'].  A new design will be added to the project.  
INFO 06:05PM [connect_design]: 	Opened active design
	Design:    flip6um_V5 [Solution type: Eigenmode]
WARNING 06:05PM [connect_setup]: 	No design setup detected.
WARNING 06:05PM [connect_setup]: 	Creating eigenmode default setup.
INFO 06:05PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.HfssEMSetup'>)


In [37]:
hfss.close()

Warning! 6 COM references still alive
Ansys will likely refuse to shut down


True

# LOM Analysis

In [30]:
from qiskit_metal.analyses.quantization import LOManalysis
c1 = LOManalysis(design, "q3d")


In [31]:
c1.sim.setup.max_passes = 10
c1.sim.setup_update(freq_ghz='5.39',solution_order = 'Medium', auto_increase_solution_order = 'False')
c1.sim.setup

{'name': 'Setup',
 'reuse_selected_design': True,
 'reuse_setup': True,
 'freq_ghz': '5.39',
 'save_fields': False,
 'enabled': True,
 'max_passes': 10,
 'min_passes': 2,
 'min_converged_passes': 2,
 'percent_error': 0.5,
 'percent_refinement': 30,
 'auto_increase_solution_order': 'False',
 'solution_order': 'Medium',
 'solver_type': 'Iterative'}

In [34]:
#c1.sim.run(components=['Q0','c0'], open_terminations=[('c0', 'in')])
c1.sim._get_results_from_renderer()
c1.sim.capacitance_matrix

INFO 06:19PM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmppbi0_bvb.txt, C, , Setup:LastAdaptive, "Original", "ohm", "nH", "fF", "mSie", 5390000000, Maxwell, 1, False
INFO 06:19PM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmpxdjerxgi.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5390000000, Maxwell, 1, False
INFO 06:19PM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmp8xsjoivy.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5390000000, Maxwell, 2, False
INFO 06:19PM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmpqefpqlaa.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5390000000, Maxwell, 3, False
INFO 06:19PM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmp5zobfguc.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5390000000, Maxwell, 4, False
INFO 06:19

,groun,pad_bot_Q0,pad_top_Q0,quad_coupler_c0
groun,305.910,-91.77400,-74.732,-78.54400
pad_bot_Q0,-91.774,106.35000,-12.893,-0.14853
pad_top_Q0,-74.732,-12.89300,106.170,-16.99400
quad_coupler_c0,-78.544,-0.14853,-16.994,95.70300


In [35]:
c1.setup.junctions = Dict({'Lj': 11, 'Cj': 2})
c1.setup.freq_readout = 7.16
c1.setup.freq_bus = []
c1.run_lom()
c1.lumped_oscillator_all

[1, 2] [3]
Predicted Values

Transmon Properties
f_Q 5.816105 [GHz]
EC 318.562662 [MHz]
EJ 14.854145 [GHz]
alpha -368.196662 [MHz]
dispersion 22.653579 [KHz]
Lq 10.995564 [nH]
Cq 60.805080 [fF]
T1 25.608410 [us]

**Coupling Properties**

tCqbus1 -8.431907 [fF]
gbus1_in_MHz -139.672104 [MHz]
χ_bus1 -6.331436 [MHz]
1/T1bus1 6214.948170 [Hz]
T1bus1 25.608410 [us]
Bus-Bus Couplings


,fQ,EC,EJ,alpha,dispersion,gbus,chi_in_MHz,χr MHz,gr MHz
1,5.902901,328.791638,14.854145,-381.15221,30.328979,[-160.0722024716805],[-9.602321424986815],9.602321,160.072202
2,6.033007,344.472623,14.854145,-401.14634,46.223241,[-167.46418635962974],[-13.197811453023212],13.197811,167.464186
3,5.849683,322.497911,14.854145,-373.172776,25.387143,[-146.95952476702186],[-7.405207555629873],7.405208,146.959525
4,5.843682,321.792618,14.854145,-372.280192,24.877932,[-143.07347049499467],[-6.949742816966789],6.949743,143.073470
5,5.844229,321.856854,14.854145,-372.361473,24.923954,[-142.87356523940386],[-6.936570949643226],6.936571,142.873565
6,5.825636,319.67685,14.854145,-369.604526,23.401275,[-140.11026150073425],[-6.47078918835781],6.470789,140.110262
7,5.822732,319.337086,14.854145,-369.175122,23.171116,[-138.86731495413363],[-6.326479351105629],6.326479,138.867315
8,5.820479,319.073745,14.854145,-368.842356,22.994031,[-138.26016540424234],[-6.248332874284189],6.248333,138.260165
9,5.816105,318.562662,14.854145,-368.196662,22.653579,[-139.67210358773642],[-6.331435570116269],6.331436,139.672104


In [36]:
c1.sim.close()

Warning! 6 COM references still alive
Ansys will likely refuse to shut down
